In [2]:
import torch
from torch import nn
import torchvision
import matplotlib.pyplot as plt

In [3]:
DATA_ROOT_DIR="FashionMnist"

In [5]:
train_data=torchvision.datasets.FashionMNIST(
root=DATA_ROOT_DIR,
train=True,
download=True,
transform=torchvision.transforms.ToTensor(),)
test_data=torchvision.datasets.FashionMNIST(
root=DATA_ROOT_DIR,
train=False,
download=True,
transform=torchvision.transforms.ToTensor(),)